In [15]:
import pymongo

In [16]:
client = pymongo.MongoClient("mongodb://13.209.69.125:27017/")
client

MongoClient(host=['13.209.69.125:27017'], document_class=dict, tz_aware=False, connect=True)

직방용 데이터베이스 및 컬렉션 생성

In [17]:
zigbang = client.crawling.zigbang
zigbang

Collection(Database(MongoClient(host=['13.209.69.125:27017'], document_class=dict, tz_aware=False, connect=True), 'crawling'), 'zigbang')

In [18]:
%%writefile zigbang.py

import geohash2
import requests
import pandas as pd

# 함수로 만들기    
def oneroom(addr):
    
    # 1. 동이름으로 위도 경도 구하기
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    lat, lng = response.json()["items"][0]["lat"], response.json()["items"][0]["lng"]
    
    # 2. 위도 경도로 geohash 알아내기
    geohash = geohash2.encode(lat, lng, precision=5) 
    
    # 3. geohash로 매물 리스트 가져오기
    url = "https://apis.zigbang.com/v2/items?\
deposit_gteq=0&domain=zigbang&geohash={}&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸".format(geohash)
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    # 4. id로 매물 정보 가져오기
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "false",
        "item_ids": ids[:900],
    }

    response = requests.post(url, params)
    items = response.json()["items"]
    return [item for item in items if addr in item["address"]]

Overwriting zigbang.py


In [19]:
import zigbang as zb

In [20]:
items = zb.oneroom("성수동")
len(items)

6

`[{}, {}, {}, ...]` json 형태
`2.8.1`

In [21]:
zigbang.insert_many(items)
# zigbang.insert(items)

* 데이터 프레임을 MongoDB에 저장

In [22]:
import pandas as pd

df_sungsu = pd.DataFrame(items)
df_sungsu.columns

Index(['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'address', 'random_location', 'is_zzim',
       'status', 'service_type', 'tags', 'address1', 'address2', 'address3',
       'manage_cost', 'reg_date', 'is_new', '_id'],
      dtype='object')

In [23]:
column_names = ["item_id", "floor", "title", "deposit", "rent"]
df_sungsu = df_sungsu[column_names]
df_sungsu

,item_id,floor,title,deposit,rent
0,29469829,반지하,"서울숲옆에 있는 분리형원룸,넓어요",500,40
1,29738029,반지하,💖 서울숲역 저렴 풀옵션 올수리 투룸,1000,55
2,29744973,반지하,"♣ 서울숲길, 풀옵션2룸 , 깨끗한 반지하, 빠른입주가능",5000,30
3,29773115,5,💖 신축형 9평 분리형 풀옵션 주차 엘베 뚝섬역 2분,2000,80
4,29738894,2,"♣ 3룸, 주차가능, 채광좋음, 뚝섬역과 서울숲역 이용가능",10000,50
5,29548099,2,"뚝섬역,귀한 투룸전세, 규모있게 넓은 거실, 대출가능",13000,50


In [24]:
zigbang = client.crawling.zigbang

In [25]:
# dataframe을 json dict화 시키기
insert_items = df_sungsu.to_dict("records")
insert_items

[{'item_id': 29469829,
  'floor': '반지하',
  'title': '서울숲옆에 있는 분리형원룸,넓어요',
  'deposit': 500,
  'rent': 40},
 {'item_id': 29738029,
  'floor': '반지하',
  'title': '💖 서울숲역 저렴 풀옵션 올수리 투룸',
  'deposit': 1000,
  'rent': 55},
 {'item_id': 29744973,
  'floor': '반지하',
  'title': '♣ 서울숲길, 풀옵션2룸 , 깨끗한 반지하, 빠른입주가능',
  'deposit': 5000,
  'rent': 30},
 {'item_id': 29773115,
  'floor': '5',
  'title': '💖 신축형 9평 분리형 풀옵션 주차 엘베  뚝섬역 2분',
  'deposit': 2000,
  'rent': 80},
 {'item_id': 29738894,
  'floor': '2',
  'title': '♣ 3룸, 주차가능, 채광좋음, 뚝섬역과 서울숲역 이용가능',
  'deposit': 10000,
  'rent': 50},
 {'item_id': 29548099,
  'floor': '2',
  'title': '뚝섬역,귀한 투룸전세, 규모있게 넓은 거실, 대출가능',
  'deposit': 13000,
  'rent': 50}]

In [26]:
zigbang2.insert_many(insert_items)

데이터 조회하기 - find()

In [27]:
results = zigbang.find()
pd.DataFrame(results)

,_id,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,61b6a1dc969551ecf4c680b5,None,29469829,https://ic.zigbang.com/ic/items/29469829/1.jpg,월세,월세,500,40,23.14,"{'m2': 23.14, 'p': '7'}",...,False,True,원룸,[추천],서울시 성동구 성수동1가,None,None,0,2021-11-30T13:20:51+09:00,False
1,61b6a1dc969551ecf4c680b6,None,29738029,https://ic.zigbang.com/ic/items/29738029/1.jpg,월세,월세,1000,55,33.06,"{'m2': 33.06, 'p': '10'}",...,False,True,빌라,[추천],서울시 성동구 성수동1가,None,None,0,2021-12-11T17:44:24+09:00,True
2,61b6a1dc969551ecf4c680b7,None,29744973,https://ic.zigbang.com/ic/items/29744973/1.jpg,월세,월세,5000,30,36.36,"{'m2': 36.36, 'p': '11'}",...,False,True,빌라,[],서울시 성동구 성수동1가,None,None,0,2021-12-09T13:15:15+09:00,False
3,61b6a1dc969551ecf4c680b8,None,29773115,https://ic.zigbang.com/ic/items/29773115/1.jpg,월세,월세,2000,80,29.75,"{'m2': 29.75, 'p': '9'}",...,False,True,원룸,[],서울시 성동구 성수동1가,None,None,7,2021-12-11T13:48:58+09:00,True
4,61b6a1dc969551ecf4c680b9,None,29738894,https://ic.zigbang.com/ic/items/29738894/1.jpg,월세,월세,10000,50,72.79,"{'m2': 72.79, 'p': '22'}",...,False,True,빌라,[],서울시 성동구 성수동1가,None,None,0,2021-12-08T21:10:45+09:00,False
5,61b6a1dc969551ecf4c680ba,None,29548099,https://ic.zigbang.com/ic/items/29548099/1.jpg,월세,월세,13000,50,52.89,"{'m2': 52.89, 'p': '16'}",...,False,True,빌라,[추천],서울시 성동구 성수동1가,None,None,0,2021-11-24T11:19:12+09:00,False


In [28]:
QUERY = { "deposit" : {"$gte" : 13000}}
results = zigbang.find(QUERY)
pd.DataFrame(results)

,_id,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,61b6a1dc969551ecf4c680ba,None,29548099,https://ic.zigbang.com/ic/items/29548099/1.jpg,월세,월세,13000,50,52.89,"{'m2': 52.89, 'p': '16'}",...,False,True,빌라,[추천],서울시 성동구 성수동1가,None,None,0,2021-11-24T11:19:12+09:00,False
